In [1]:
import os

In [2]:
%pwd

'd:\\industry standard ml project\\Redwine-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\industry standard ml project\\Redwine-project'

#### Creating entity
- Entity is the custom return type of function

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 

#### Updating ConfigurationManager
- ConfigurationManager will give you all the filepath , all the source URL. It will return everything in one shot

In [6]:
from mlProject.constants import *
from mlProject.utils.comon import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        
        return data_ingestion_config
    

#### Updateing data ingestion components

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.comon import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloa with following info: \n{headers}")
        else:
            logger.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts th zip file into the data direcotry 
        Function return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-18 18:47:32,571: INFO: comon: yaml file: config\config.yaml loaded successfully]
[2024-04-18 18:47:32,577: INFO: comon: yaml file: params.yaml loaded successfully]
[2024-04-18 18:47:32,589: INFO: comon: yaml file: schema.yaml loaded successfully]
[2024-04-18 18:47:32,594: INFO: comon: crated directory at: artifacts]
[2024-04-18 18:47:32,599: INFO: comon: crated directory at: artifacts/data_ingestion]
[2024-04-18 18:47:36,214: INFO: 2780010455: artifacts/data_ingestion/data.zip downloa with following info: 
Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 727E:3587CD:5A722B:75F11E:662119EB
Accept-Ranges: bytes
Date: Thu, 18 Apr 2024